# Can a card's attributes be used to predict its price?

In [88]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import random

In [228]:
# random.seed(42)

# all_sets = pd.read_csv("set_info.csv")
# major_sets = all_sets[all_sets["set_size"] > 100]
# sample = major_sets.sample(1)
# print(major_sets)
# incomplete = major_sets.drop([3,5,7,11,12,21,22,23,25,26,28,35,44,47,58,69,81,87])

main_sets = pd.read_csv(".\data\main_set_info.csv")
# main_sets = main_sets[73:]
# main_sets

In [220]:
for set_, size in zip(main_sets["set_id"], main_sets["set_size"]):
    names = []
    supertypes = []
    subtypes = []
    sets = []
    rarities = []
    cmcs = []
    colors = []
    modern_legalities = []
    standard_legalities = []
    commander_legalities = []
    reprints = []
    artists = []
    prices = []
    stop = False
    for card_num in range(size):
        print(set_, card_num+1)
        try:
            response = requests.get('https://scryfall.com/card/{}/{}/'.format(set_, str(card_num+1)))
            html = BeautifulSoup(response.content, 'html.parser')

            card_title = html.find("h1", class_ = "card-text-title").text

            try:
                card_has_suspend = "Suspend" in html.find("div", class_ = "card-text-oracle").text.strip()
            except AttributeError:
                card_has_suspend = False        
            if card_has_suspend == True:
                continue
            

            card_name = card_title.strip().split("\n")[0]
            if card_name == "Evermind":
                continue
            names.append(card_name)

            card_type = html.find("p", class_ = "card-text-type-line").text.strip()
            if "—" in card_type:
                card_types = card_type.split("—")
                card_supertype = card_types[0].strip()
                card_subtype = card_types[1].strip()   
                supertypes.append(card_supertype)
                subtypes.append(card_subtype)
            elif "—" not in card_type:
                card_supertype = card_type
                card_subtype = "None"   
                supertypes.append(card_supertype)
                subtypes.append(card_subtype)

            card_artist = html.find("p", class_ = "card-text-artist").text.strip().split("\n")[1].strip()
            artists.append(card_artist.encode('utf-8'))

            card_current_prints = html.find("div", class_ = "prints-current").text.strip().replace("\n", "").split("              ")
            card_set = card_current_prints[0]
            wrds = card_set.strip().split(" ")[0:-1]
            card_set = " ".join(wrds)
            sets.append(card_set)
            card_rarity = card_current_prints[1].split(" · ")[1]
            rarities.append(card_rarity)

            card_legalities = []
            for j in html.find_all("div", class_ = "card-legality-item"):
                card_legalities.append(j.text.split())
            for l in card_legalities:
                if l[0] == "Standard":
                    if (l[1] == "Not") | (l[1] == "Banned"):
                        card_standard_legal = False
                    elif l[1] == "Legal":
                        card_standard_legal = True
                if l[0] == "Modern":
                    if (l[1] == "Not") | (l[1] == "Banned"):
                        card_modern_legal = False
                    elif l[1] == "Legal":
                        card_modern_legal = True
                if l[0] == "Commander":
                    if (l[1] == "Not") | (l[1] == "Banned"):
                        card_commander_legal = False
                    elif l[1] == "Legal":
                        card_commander_legal = True
            standard_legalities.append(card_standard_legal)
            modern_legalities.append(card_modern_legal)
            commander_legalities.append(card_commander_legal)

            for k in html.find_all("tr", class_ = "current"):
                if len(re.findall("\d\.\d\d", k.text.strip())) != 0:
                    card_prices = re.findall("\d?\d\.\d\d", k.text.strip())
                    card_price = float(card_prices[0])
            prices.append(card_price)

            l = []
            for z in html.select("div[class = prints] > table[class=prints-table]"):
                l.append(z.text)
            for n in l:
                sections = re.sub("  {1,}", " ", n.replace("\n", ""))
                if "Prints" in sections:
                    if len(sections.split()) > 6:
                        card_is_reprint = True
                    else:
                        card_is_reprint = False
                else:
                    continue       
            reprints.append(card_is_reprint)

            if (card_has_suspend == False) & ("Land" not in card_supertype):
                card_casting_cost = re.findall("[A-Z1-9]", card_title.strip().split("\n")[1].strip())
                try: 
                    if len(card_casting_cost) == 0:
                        card_cmc = 0
                    else:
                        generic_mana_count = int(card_casting_cost[0])
                        card_cmc = generic_mana_count + len(card_casting_cost[1:])
                except ValueError:
                    card_cmc = len(card_casting_cost)

                card_color_list = []
                if "W" in card_casting_cost:
                    card_color_list.append("White")
                if "U" in card_casting_cost:
                    card_color_list.append("Blue")
                if "B" in card_casting_cost:
                    card_color_list.append("Black")
                if "R" in card_casting_cost:
                    card_color_list.append("Red")
                if "G" in card_casting_cost:
                    card_color_list.append("Green")
                if len(card_color_list) == 1:
                    card_color = card_color_list[0]
                if len(card_color_list) > 1:
                    card_color = "Gold"
                if len(card_color_list) == 0:
                    card_color = "Colorless"
                cmcs.append(card_cmc)
                colors.append(card_color)

            elif (card_has_suspend == False) & (("Land" in card_supertype)):
                card_cmc = 0
                card_color = "Colorless"
                cmcs.append(card_cmc)
                colors.append(card_color)
        except AttributeError as e:
#             if e == "'NoneType' object has no attribute 'text'":
            print(e)
            continue
#             elif e != "'NoneType' object has no attribute 'text'":
#                 print("other", e)
#                 stop = True
#                 continue
#     if stop == True:
#         break
    data = {"names": names, "supertypes": supertypes, "subtypes": subtypes, "set": sets, "rarity": rarities, "cmc": cmcs, "color": colors, "modern legality": modern_legalities, "standard legality": standard_legalities, "commander legality": commander_legalities, "reprint": reprints, "artist": artists, "price": prices}
    df = pd.DataFrame(data)
    df.to_csv(".\data\main_sets\{}_info.csv".format(set_), index = False)

sok 1
sok 2
sok 3
sok 4
sok 5
sok 6
sok 7
sok 8
sok 9
sok 10
sok 11
sok 12
sok 13
sok 14
sok 15
sok 16
sok 17
sok 18
sok 19
sok 20
sok 21
sok 22
sok 23
sok 24
sok 25
sok 26
sok 27
sok 28
sok 29
sok 30
sok 31
sok 32
sok 33
sok 34
sok 35
sok 36
sok 37
sok 38
sok 39
sok 40
sok 41
sok 42
sok 43
sok 44
sok 45
sok 46
sok 47
sok 48
sok 49
sok 50
sok 51
sok 52
sok 53
sok 54
sok 55
sok 56
sok 57
sok 58
sok 59
sok 60
sok 61
sok 62
sok 63
sok 64
sok 65
sok 66
sok 67
sok 68
sok 69
sok 70
sok 71
sok 72
sok 73
sok 74
sok 75
sok 76
sok 77
sok 78
sok 79
sok 80
sok 81
sok 82
sok 83
sok 84
sok 85
sok 86
sok 87
sok 88
sok 89
sok 90
sok 91
sok 92
sok 93
sok 94
sok 95
sok 96
sok 97
sok 98
sok 99
sok 100
sok 101
sok 102
sok 103
sok 104
sok 105
sok 106
sok 107
sok 108
sok 109
sok 110
sok 111
sok 112
sok 113
sok 114
sok 115
sok 116
sok 117
sok 118
sok 119
sok 120
sok 121
sok 122
sok 123
sok 124
sok 125
sok 126
sok 127
sok 128
sok 129
sok 130
sok 131
sok 132
sok 133
sok 134
sok 135
sok 136
sok 137
sok 138
sok 

mrd 129
mrd 130
mrd 131
mrd 132
mrd 133
mrd 134
mrd 135
mrd 136
mrd 137
mrd 138
mrd 139
mrd 140
mrd 141
mrd 142
mrd 143
mrd 144
mrd 145
mrd 146
mrd 147
mrd 148
mrd 149
mrd 150
mrd 151
mrd 152
mrd 153
mrd 154
mrd 155
mrd 156
mrd 157
mrd 158
mrd 159
mrd 160
mrd 161
mrd 162
mrd 163
mrd 164
mrd 165
mrd 166
mrd 167
mrd 168
mrd 169
mrd 170
mrd 171
mrd 172
mrd 173
mrd 174
mrd 175
mrd 176
mrd 177
mrd 178
mrd 179
mrd 180
mrd 181
mrd 182
mrd 183
mrd 184
mrd 185
mrd 186
mrd 187
mrd 188
mrd 189
mrd 190
mrd 191
mrd 192
mrd 193
mrd 194
mrd 195
mrd 196
mrd 197
mrd 198
mrd 199
mrd 200
mrd 201
mrd 202
mrd 203
mrd 204
mrd 205
mrd 206
mrd 207
mrd 208
mrd 209
mrd 210
mrd 211
mrd 212
mrd 213
mrd 214
mrd 215
mrd 216
mrd 217
mrd 218
mrd 219
mrd 220
mrd 221
mrd 222
mrd 223
mrd 224
mrd 225
mrd 226
mrd 227
mrd 228
mrd 229
mrd 230
mrd 231
mrd 232
mrd 233
mrd 234
mrd 235
mrd 236
mrd 237
mrd 238
mrd 239
mrd 240
mrd 241
mrd 242
mrd 243
mrd 244
mrd 245
mrd 246
mrd 247
mrd 248
mrd 249
mrd 250
mrd 251
mrd 252
mrd 253


tor 134
tor 135
tor 136
tor 137
tor 138
tor 139
tor 140
tor 141
tor 142
tor 143
ody 1
ody 2
ody 3
ody 4
ody 5
ody 6
ody 7
ody 8
ody 9
ody 10
ody 11
ody 12
ody 13
ody 14
ody 15
ody 16
ody 17
ody 18
ody 19
ody 20
ody 21
ody 22
ody 23
ody 24
ody 25
ody 26
ody 27
ody 28
ody 29
ody 30
ody 31
ody 32
ody 33
ody 34
ody 35
ody 36
ody 37
ody 38
ody 39
ody 40
ody 41
ody 42
ody 43
ody 44
ody 45
ody 46
ody 47
ody 48
ody 49
ody 50
ody 51
ody 52
ody 53
ody 54
ody 55
ody 56
ody 57
ody 58
ody 59
ody 60
ody 61
ody 62
ody 63
ody 64
ody 65
ody 66
ody 67
ody 68
ody 69
ody 70
ody 71
ody 72
ody 73
ody 74
ody 75
ody 76
ody 77
ody 78
ody 79
ody 80
ody 81
ody 82
ody 83
ody 84
ody 85
ody 86
ody 87
ody 88
ody 89
ody 90
ody 91
ody 92
ody 93
ody 94
ody 95
ody 96
ody 97
ody 98
ody 99
ody 100
ody 101
ody 102
ody 103
ody 104
ody 105
ody 106
ody 107
ody 108
ody 109
ody 110
ody 111
ody 112
ody 113
ody 114
ody 115
ody 116
ody 117
ody 118
ody 119
ody 120
ody 121
ody 122
ody 123
ody 124
ody 125
ody 126
ody 127
ody 128
ody 

pcy 75
pcy 76
pcy 77
pcy 78
pcy 79
pcy 80
pcy 81
pcy 82
pcy 83
pcy 84
pcy 85
pcy 86
pcy 87
pcy 88
pcy 89
pcy 90
pcy 91
pcy 92
pcy 93
pcy 94
pcy 95
pcy 96
pcy 97
pcy 98
pcy 99
pcy 100
pcy 101
pcy 102
pcy 103
pcy 104
pcy 105
pcy 106
pcy 107
pcy 108
pcy 109
pcy 110
pcy 111
pcy 112
pcy 113
pcy 114
pcy 115
pcy 116
pcy 117
pcy 118
pcy 119
pcy 120
pcy 121
pcy 122
pcy 123
pcy 124
pcy 125
pcy 126
pcy 127
pcy 128
pcy 129
pcy 130
pcy 131
pcy 132
pcy 133
pcy 134
pcy 135
pcy 136
pcy 137
pcy 138
pcy 139
pcy 140
pcy 141
pcy 142
pcy 143
nem 1
nem 2
nem 3
nem 4
nem 5
nem 6
nem 7
nem 8
nem 9
nem 10
nem 11
nem 12
nem 13
nem 14
nem 15
nem 16
nem 17
nem 18
nem 19
nem 20
nem 21
nem 22
nem 23
nem 24
nem 25
nem 26
nem 27
nem 28
nem 29
nem 30
nem 31
nem 32
nem 33
nem 34
nem 35
nem 36
nem 37
nem 38
nem 39
nem 40
nem 41
nem 42
nem 43
nem 44
nem 45
nem 46
nem 47
nem 48
nem 49
nem 50
nem 51
nem 52
nem 53
nem 54
nem 55
nem 56
nem 57
nem 58
nem 59
nem 60
nem 61
nem 62
nem 63
nem 64
nem 65
nem 66
nem 67
nem 68
nem 69

usg 248
usg 249
usg 250
usg 251
usg 252
usg 253
usg 254
usg 255
usg 256
usg 257
usg 258
usg 259
usg 260
usg 261
usg 262
usg 263
usg 264
usg 265
usg 266
usg 267
usg 268
usg 269
usg 270
usg 271
usg 272
usg 273
usg 274
usg 275
usg 276
usg 277
usg 278
usg 279
usg 280
usg 281
usg 282
usg 283
usg 284
usg 285
usg 286
usg 287
usg 288
usg 289
usg 290
usg 291
usg 292
usg 293
usg 294
usg 295
usg 296
usg 297
usg 298
usg 299
usg 300
usg 301
usg 302
usg 303
usg 304
usg 305
usg 306
usg 307
usg 308
usg 309
usg 310
usg 311
usg 312
usg 313
usg 314
usg 315
usg 316
usg 317
usg 318
usg 319
usg 320
usg 321
usg 322
usg 323
usg 324
usg 325
usg 326
usg 327
usg 328
usg 329
usg 330
usg 331
usg 332
usg 333
usg 334
usg 335
usg 336
usg 337
usg 338
usg 339
usg 340
usg 341
usg 342
usg 343
usg 344
usg 345
usg 346
usg 347
usg 348
usg 349
usg 350
exo 1
exo 2
exo 3
exo 4
exo 5
exo 6
exo 7
exo 8
exo 9
exo 10
exo 11
exo 12
exo 13
exo 14
exo 15
exo 16
exo 17
exo 18
exo 19
exo 20
exo 21
exo 22
exo 23
exo 24
exo 25
exo 26
exo

mir 24
mir 25
mir 26
mir 27
mir 28
mir 29
mir 30
mir 31
mir 32
mir 33
mir 34
mir 35
mir 36
mir 37
mir 38
mir 39
mir 40
mir 41
mir 42
mir 43
mir 44
mir 45
mir 46
mir 47
mir 48
mir 49
mir 50
mir 51
mir 52
mir 53
mir 54
mir 55
mir 56
mir 57
mir 58
mir 59
mir 60
mir 61
mir 62
mir 63
mir 64
mir 65
mir 66
mir 67
mir 68
mir 69
mir 70
mir 71
mir 72
mir 73
mir 74
mir 75
mir 76
mir 77
mir 78
mir 79
mir 80
mir 81
mir 82
mir 83
mir 84
mir 85
mir 86
mir 87
mir 88
mir 89
mir 90
mir 91
mir 92
mir 93
mir 94
mir 95
mir 96
mir 97
mir 98
mir 99
mir 100
mir 101
mir 102
mir 103
mir 104
mir 105
mir 106
mir 107
mir 108
mir 109
mir 110
mir 111
mir 112
mir 113
mir 114
mir 115
mir 116
mir 117
mir 118
mir 119
mir 120
mir 121
mir 122
mir 123
mir 124
mir 125
mir 126
mir 127
mir 128
mir 129
mir 130
mir 131
mir 132
mir 133
mir 134
mir 135
mir 136
mir 137
mir 138
mir 139
mir 140
mir 141
mir 142
mir 143
mir 144
mir 145
mir 146
mir 147
mir 148
mir 149
mir 150
mir 151
mir 152
mir 153
mir 154
mir 155
mir 156
mir 157
mir 

'NoneType' object has no attribute 'text'
all 192
'NoneType' object has no attribute 'text'
all 193
'NoneType' object has no attribute 'text'
all 194
'NoneType' object has no attribute 'text'
all 195
'NoneType' object has no attribute 'text'
all 196
'NoneType' object has no attribute 'text'
all 197
'NoneType' object has no attribute 'text'
all 198
'NoneType' object has no attribute 'text'
all 199
'NoneType' object has no attribute 'text'
hml 1
hml 2
'NoneType' object has no attribute 'text'
hml 3
'NoneType' object has no attribute 'text'
hml 4
hml 5
hml 6
hml 7
hml 8
hml 9
hml 10
'NoneType' object has no attribute 'text'
hml 11
hml 12
hml 13
'NoneType' object has no attribute 'text'
hml 14
hml 15
hml 16
hml 17
hml 18
hml 19
'NoneType' object has no attribute 'text'
hml 20
hml 21
hml 22
hml 23
hml 24
hml 25
'NoneType' object has no attribute 'text'
hml 26
hml 27
'NoneType' object has no attribute 'text'
hml 28
hml 29
hml 30
'NoneType' object has no attribute 'text'
hml 31
hml 32
'NoneTy

'NoneType' object has no attribute 'text'
fem 69
fem 70
fem 71
'NoneType' object has no attribute 'text'
fem 72
'NoneType' object has no attribute 'text'
fem 73
fem 74
'NoneType' object has no attribute 'text'
fem 75
fem 76
fem 77
fem 78
fem 79
fem 80
'NoneType' object has no attribute 'text'
fem 81
fem 82
fem 83
fem 84
fem 85
fem 86
fem 87
fem 88
fem 89
fem 90
fem 91
fem 92
fem 93
fem 94
fem 95
fem 96
fem 97
fem 98
fem 99
fem 100
fem 101
fem 102
fem 103
'NoneType' object has no attribute 'text'
fem 104
'NoneType' object has no attribute 'text'
fem 105
'NoneType' object has no attribute 'text'
fem 106
'NoneType' object has no attribute 'text'
fem 107
'NoneType' object has no attribute 'text'
fem 108
'NoneType' object has no attribute 'text'
fem 109
'NoneType' object has no attribute 'text'
fem 110
'NoneType' object has no attribute 'text'
fem 111
'NoneType' object has no attribute 'text'
fem 112
'NoneType' object has no attribute 'text'
fem 113
'NoneType' object has no attribute 'text'

atq 42
atq 43
atq 44
atq 45
atq 46
atq 47
atq 48
atq 49
atq 50
atq 51
atq 52
atq 53
atq 54
atq 55
atq 56
atq 57
atq 58
atq 59
atq 60
atq 61
atq 62
atq 63
atq 64
atq 65
atq 66
atq 67
atq 68
atq 69
atq 70
atq 71
atq 72
atq 73
atq 74
atq 75
atq 76
atq 77
atq 78
atq 79
atq 80
'NoneType' object has no attribute 'text'
atq 81
atq 82
'NoneType' object has no attribute 'text'
atq 83
'NoneType' object has no attribute 'text'
atq 84
'NoneType' object has no attribute 'text'
atq 85
'NoneType' object has no attribute 'text'
atq 86
'NoneType' object has no attribute 'text'
atq 87
'NoneType' object has no attribute 'text'
atq 88
'NoneType' object has no attribute 'text'
atq 89
'NoneType' object has no attribute 'text'
atq 90
'NoneType' object has no attribute 'text'
atq 91
'NoneType' object has no attribute 'text'
atq 92
'NoneType' object has no attribute 'text'
atq 93
'NoneType' object has no attribute 'text'
atq 94
'NoneType' object has no attribute 'text'
atq 95
'NoneType' object has no attribute

In [146]:
# response = requests.get('https://scryfall.com/card/lrw/257/')
# html = BeautifulSoup(response.content, 'html.parser')

In [152]:
# card_title = html.find("h1", class_ = "card-text-title").text

# try:
#     card_has_suspend = "Suspend" in html.find("div", class_ = "card-text-oracle").text.strip()
# except AttributeError:
#     card_has_suspend = False
# # if card_has_suspend == True:
# #     continue


# card_name = card_title.strip().split("\n")[0]
# print(card_name)

# card_type = html.find("p", class_ = "card-text-type-line").text.strip()
# if "—" in card_type:
#     card_types = card_type.split("—")
#     card_supertype = card_types[0].strip()
#     card_subtype = card_types[1].strip()   
#     print(card_supertype)
#     print(card_subtype)
# elif "—" not in card_type:
#     card_supertype = card_type
#     card_subtype = "None"   
#     print(card_supertype)
#     print(card_subtype)

# card_artist = html.find("p", class_ = "card-text-artist").text.strip().split("\n")[1].strip()
# print(card_artist.encode('utf-8'))

# card_current_prints = html.find("div", class_ = "prints-current").text.strip().replace("\n", "").split("              ")
# card_set = card_current_prints[0]
# wrds = card_set.strip().split(" ")[0:-1]
# card_set = " ".join(wrds)
# print(card_set)
# card_rarity = card_current_prints[1].split(" · ")[1]
# print(card_rarity)

# card_legalities = []
# for j in html.find_all("div", class_ = "card-legality-item"):
#     card_legalities.append(j.text.split())
# for l in card_legalities:
#     if l[0] == "Standard":
#         if (l[1] == "Not") | (l[1] == "Banned"):
#             card_standard_legal = False
#         elif l[1] == "Legal":
#             card_standard_legal = True
#     if l[0] == "Modern":
#         if (l[1] == "Not") | (l[1] == "Banned"):
#             card_modern_legal = False
#         elif l[1] == "Legal":
#             card_modern_legal = True
#     if l[0] == "Commander":
#         if (l[1] == "Not") | (l[1] == "Banned"):
#             card_commander_legal = False
#         elif l[1] == "Legal":
#             card_commander_legal = True
# print("sl", card_standard_legal)
# print("ml", card_modern_legal)
# print("cl", card_commander_legal)

# for k in html.find_all("tr", class_ = "current"):
#     if len(re.findall("\d\.\d\d", k.text.strip())) != 0:
#         card_prices = re.findall("\d?\d\.\d\d", k.text.strip())
#         card_price = float(card_prices[0])
# print(card_price)

# l = []
# for z in html.select("div[class = prints] > table[class=prints-table]"):
#     l.append(z.text)
# for n in l:
#     sections = re.sub("  {1,}", " ", n.replace("\n", ""))
#     if "Prints" in sections:
#         if len(sections.split()) > 6:
#             card_is_reprint = True
#         else:
#             card_is_reprint = False
#     else:
#         continue       
# print("rp", card_is_reprint)

# if (card_has_suspend == False) & ("Land" not in card_supertype):
#     card_casting_cost = re.findall("[A-Z1-9]", card_title.strip().split("\n")[1].strip())
#     try: 
#         if len(card_casting_cost) == 0:
#             card_cmc = 0
#         else:
#             generic_mana_count = int(card_casting_cost[0])
#             card_cmc = generic_mana_count + len(card_casting_cost[1:])
#     except ValueError:
#         card_cmc = len(card_casting_cost)

#     card_color_list = []
#     if "W" in card_casting_cost:
#         card_color_list.append("White")
#     if "U" in card_casting_cost:
#         card_color_list.append("Blue")
#     if "B" in card_casting_cost:
#         card_color_list.append("Black")
#     if "R" in card_casting_cost:
#         card_color_list.append("Red")
#     if "G" in card_casting_cost:
#         card_color_list.append("Green")
#     if len(card_color_list) == 1:
#         card_color = card_color_list[0]
#     if len(card_color_list) > 1:
#         card_color = "Gold"
#     if len(card_color_list) == 0:
#         card_color = "Colorless"
#     print(card_cmc)
#     print(card_color)

# elif (card_has_suspend == False) & (("Land" in card_supertype)):
#     card_cmc = 0
#     card_color = "Colorless"
#     print(card_cmc)

#     print(card_color)

Herbal Poultice
Artifact
None
b'Scott Hampton'
Lorwyn
Common
sl False
ml True
cl True
0.24
rp False
0
Colorless


In [238]:
cols = ["names", "supertypes", "subtypes", "set", "rarity", "cmc", "color", "modern legality", "standard legality", "commander legality", "reprint", "artist", "price"]
all_sets_df = pd.DataFrame(columns = cols)
for set_ in main_sets["set_id"]:
    all_sets_df = pd.concat([all_sets_df, pd.read_csv("./data/main_sets/{}_info.csv".format(set_))])


In [244]:
all_sets_df.to_csv("./data/all_set_data.csv", index = False)